In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.interpolate import CubicSpline
import gym
import time
import matplotlib.pyplot as plt
%matplotlib inline

/Users/captain/opt/anaconda3/lib/python3.8/site-packages/ale_py/roms/__init__.py:94: DeprecationWarning: Automatic importing of atari-py roms won't be supported in future releases of ale-py. Please migrate over to using `ale-import-roms` OR an ALE-supported ROM package. To make this warning disappear you can run `ale-import-roms --import-from-pkg atari_py.atari_roms`.For more information see: https://github.com/mgbellemare/Arcade-Learning-Environment#rom-management
  _RESOLVED_ROMS = _resolve_roms()


### 1. Epsilon-Greedy Algorithm 

In [2]:
# there are 4 slot machines, the means and standard deviations of their payoffs are listed below

means = np.array([1,2,3,4])
stds = np.array([1,2,3,4])
narms = len(means)
npull = 500


In [3]:
# when we don't know the payoff distributions we don't know to pull 4 repeatedly
# we must learn which arm is best
# but we shouldn't expect to do better than when we do know the distribution!

In [4]:
def tot_return_fun_eps_greedy_linear_anneal(epsilon_big,epsilon_small,epsilon_delta,npulls):
    avg_so_far = np.zeros(narms)
    pulls_per_arm = np.zeros(narms)
    tot_return = 0
    epsilon = epsilon_big #starting value of epsilon
    for pull in range(npulls):
        if np.random.random() < epsilon:  #exploration
            this_choice = np.random.choice(narms)
        else: #exploitation
            this_choice = np.argmax(avg_so_far)
        
        #generate return from selected arm
        payout = np.random.normal(loc=means[this_choice],scale = stds[this_choice])
        #update average return for the selected arm
        avg_so_far[this_choice] *= pulls_per_arm[this_choice]
        avg_so_far[this_choice] += payout
        pulls_per_arm[this_choice] += 1
        avg_so_far[this_choice] /= pulls_per_arm[this_choice]        
        tot_return += payout #update total return
        if epsilon > epsilon_small: #update epsilon as long as it is above the defined minimum value
            epsilon -= epsilon_delta
    return tot_return

In [5]:
# A
nsim = 5000
vals = np.zeros(nsim)
for s in range(nsim):
    vals[s] = tot_return_fun_eps_greedy_linear_anneal(1.0,0.25,0.005,npull)
np.mean(vals)

1696.6771461131782

In [6]:
# B
nsim = 5000
vals = np.zeros(nsim)
for s in range(nsim):
    vals[s] = tot_return_fun_eps_greedy_linear_anneal(0.5,0.05,0.01,npull)
np.mean(vals)

1809.443331941108

In [7]:
# C
nsim = 5000
vals = np.zeros(nsim)
for s in range(nsim):
    vals[s] = tot_return_fun_eps_greedy_linear_anneal(0.9,0.1,0.005,npull)
np.mean(vals)

1788.5421324288577

In [8]:
# D
nsim = 5000
vals = np.zeros(nsim)
for s in range(nsim):
    vals[s] = tot_return_fun_eps_greedy_linear_anneal(0.75,0.15,0.01,npull)
np.mean(vals)

1793.335176613061

In [9]:
# E
nsim = 5000
vals = np.zeros(nsim)
for s in range(nsim):
    vals[s] = tot_return_fun_eps_greedy_linear_anneal(0.15,0.15,0.0,npull)
np.mean(vals)

1752.309568740626

### 2. Non Integer Dynamic Programming by Interpolation

In [10]:
price = 1 # price per ton sold
M = 100 # current tons in mine
delta = 0.9 # discount rate
T = 15 # years with a lease

sValues = np.arange(M+1) # all possible levels of ore in mine
tValues = np.arange(T+1) # get to mine in years 0-14, in year 15 mine shuts down

sN = len(sValues) # how many s and t values are there?
tN = len(tValues)


In [11]:
V = np.zeros((sN,tN)) # value function
U = np.zeros((sN,tN)) # place to store optimal number of tons to mine

In [12]:
V[:,tN-1] = 0 # terminal condition value function.. this is a bit redundant because of lines above, but it's worth repeating!
U[:,tN-1] = 0 # terminal tons of ore to mine

In [13]:
for ti in reversed(range(tN-1)): # loop backwards in time
    # create a cubic spline for tomorrow's value function
    cs = CubicSpline(sValues,V[:,ti+1])
    for si in range(sN): # loop over all possible tons of ore in mine
        t=tValues[ti] # what is the actual time and tons in the mine
        s=sValues[si] 
        
        # create a function that defines the value function as a function of x
        def fun(x):
            return -(price*x - (x**2)/(1+s) + delta*cs(s-x))
        # do some NLP to get the optimal x
        opt = minimize(fun,[s/2.0],method='L-BFGS-B',bounds=[(0,s)],tol=1e-8)
        V[si,ti] = -opt.fun[0] # pick the one that maximizes that value
        U[si,ti] = opt.x[0]

/Users/captain/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx


In [14]:
V[100,0]

58.15776317781976

In [15]:
# 58.113941952344646 is V[100,0] from the integer dynamic programming
V[100,0]-58.113941952344646

0.04382122547511358

## Finding the pixels where the game is played

In [16]:
env = gym.make("Pong-v0") # create the pong 'environment'

/Users/captain/opt/anaconda3/lib/python3.8/site-packages/gym/envs/registration.py:505: UserWarning: WARN: The environment Pong-v0 is out of date. You should consider upgrading to version `v5` with the environment ID `ALE/Pong-v5`.
  logger.warn(
A.L.E: Arcade Learning Environment (version 0.7.4+069f8bd)
[Powered by Stella]
/Users/captain/opt/anaconda3/lib/python3.8/site-packages/gym/utils/seeding.py:138: DeprecationWarning: WARN: Function `hash_seed(seed, max_bytes)` is marked as deprecated and will be removed in the future. 
  deprecation(
/Users/captain/opt/anaconda3/lib/python3.8/site-packages/gym/utils/seeding.py:175: DeprecationWarning: WARN: Function `_bigint_from_bytes(bytes)` is marked as deprecated and will be removed in the future. 
  deprecation(


In [17]:
def prepro(I):
    # preprocess each frame for learning
    # save some memory and computation
    # pre-process the image from a 210x160x3 uint8 frame into an (80x80) float array 
    I = I[35:195,:,:].copy() # crop the top of the image...score image doesn't matter for how to play
    I = I[::2,::2,0].copy()
    I[I == 144] = 0 # erase background (background type 1)
    I[I == 109] = 0 # erase background (background type 2)
    I[I != 0] = 1 # everything else (paddles, ball) just set to 1
    return np.array(I.copy())

In [18]:
render = False
slow = False
ngames = 500
score_vec = np.zeros(ngames)
for game in range(ngames):
    print(game)
    done = False
    score = 0
    raw_pixels = env.reset()
    pixels = prepro(raw_pixels)
    reward_vec = []
    while not done:
        # the ball is between columns 10 - 69
        ball = np.where(pixels[:,10:70]==1)
        if len(ball[0])==0: # no ball on screen
            # do nothing...
            action = 0
        else: # move towards the ball
            # my paddle is in columns 70 and 71
            my_paddle = np.mean(np.where(pixels[:,71]==1))
            # move towards ball
            ball = np.mean(ball[0])
            if my_paddle < ball:
                action = 3
            elif my_paddle > ball:
                action = 2
            else:
                action = 0

        # 2 is down, 3 is up
    #     action = 2#np.random.choice([0,2,3]) # we'll eventually pick the action from a NN
        if render == True: # do you actually want to visualize the playing?
            env.render()
            if slow == True: # do you want to slow it down so you can see the game better?
                time.sleep(0.005)
        raw_pixels, reward, done, info = env.step(action)
        reward_vec.append(reward)
        pixels = prepro(raw_pixels)

        score += reward
    
    score_vec[game] = np.sum(np.array(reward_vec)==1)
    
np.mean(score_vec)
        
        
    

0


/Users/captain/opt/anaconda3/lib/python3.8/site-packages/gym/utils/seeding.py:47: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high, size, dtype])` instead.
  deprecation(


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


7.564

In [19]:
np.std(score_vec)/np.sqrt(500)

0.13279987951801764

In [20]:
np.where(pixels[:,71]==1) # paddle here

(array([63, 64, 65, 66, 67, 68, 69, 70]),)

In [21]:
np.where(pixels[:,70]==1) # paddle here

(array([63, 64, 65, 66, 67, 68, 69, 70]),)

In [22]:
np.where(pixels[:,69]==1) # no paddle here

(array([], dtype=int64),)

In [23]:
np.where(pixels[:,10]==1) # no paddle here

(array([], dtype=int64),)

In [24]:
np.where(pixels[:,9]==1) # paddle here

(array([77, 78, 79]),)